Applied Data Science Captstone Week 3

Code - Using Beautiful Soup to Scrape the URL

In [1]:
!pip3 install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
#install 
import geocoder # import geocoder


In [3]:
# Set up environment
from bs4 import BeautifulSoup
import requests
import pandas as pd

Scrape the list of postal codes of Canada using BeautifulSoup

In [4]:
#scrape postal codes
table_contents=[]
can_postal_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(can_postal_url).text

can_html  = BeautifulSoup(source, 'xml')
can_html = can_html.find('table')



Convert the HTML content to dataframe

In [5]:
for row in can_html.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
df.tail()

,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Cleaning data

1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
#  Clean NA cells and cells whose `Borough` is `Not assigned`
df = df.dropna()
df = df[df['Borough'] != 'Not assigned']

# If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough.
not_assigned_ids = df['Neighborhood'] == 'Not assigned'
df['Neighborhood'][not_assigned_ids] = df['Borough'][not_assigned_ids]

# Replace '/' by ','
df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',')
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [8]:
# Merge cells having the same postal code
temp = df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp = temp.reset_index(drop=False)
temp.rename(columns={'Neighborhood':'Neighborhood_joined'}, inplace=True)

temp.head()

,PostalCode,Neighborhood_joined
0,M1B,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [9]:
df_merge = pd.merge(df, temp, on='PostalCode')
df_merge.drop(['Neighborhood'], axis=1, inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'}, inplace=True)
df_merge.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


List number of columns and rows

In [10]:
df_merge.shape

(103, 3)

Map of postal codes and locations

In [11]:
!pip3 install folium
import folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [17]:
geo_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geo_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_merge = pd.merge(geo_df, df_merge, on='PostalCode')
geo_merge.head(100)

toronto_df = geo_merge[geo_merge['Borough'].str.contains("Toronto")]
toronto_df.reset_index(drop=True, inplace=True)

toronto_df

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M4E,43.676357,-79.293031,East Toronto,The Beaches
1,M4J,43.685347,-79.338106,East York/East Toronto,The Danforth East
2,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
3,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
4,M4M,43.659526,-79.340923,East Toronto,Studio District
5,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park
6,M4P,43.712751,-79.390197,Central Toronto,Davisville North
7,M4R,43.715383,-79.405678,Central Toronto,North Toronto West
8,M4S,43.704324,-79.388790,Central Toronto,Davisville
9,M4T,43.689574,-79.383160,Central Toronto,"Moore Park, Summerhill East"


In [20]:
# Create Toronto map
map_toronto = folium.Map(location=[43.65, -79.38], zoom_start=10)

# Add markers
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto